In [1]:
import requests
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import os
import webbrowser
import folium
from folium import plugins
print(folium.__version__)

0.14.0


In [2]:
data = pd.read_excel('2022 지역별 전력사용량.xlsx')


In [3]:
data = data.iloc[1:, :]

In [4]:
data1 = data.rename(columns=data.iloc[0])


In [5]:
data1 = data1.drop(data1.index[0]).reset_index()


In [6]:
data1 = data1.drop(['index'], axis='columns')


In [7]:
data1['1월'][1830] = 341596071
data1['1월'][1824] = 540525

In [8]:
data2 = data1[data1['계약종별'] == '합 계']


In [9]:
data2 = data2.reset_index().drop(['index'], axis='columns')

In [10]:
data2['합계'] = data2.iloc[:, 4:].sum(axis=1)
pd.options.display.float_format = '{:.5f}'.format

In [11]:
data3 = data2.copy()


In [12]:
data3[['시군구', '합계']]

,시군구,합계
0,종로구,1721240046.00000
1,중구,2305199597.00000
2,용산구,1514990326.00000
3,성동구,2235049568.00000
4,동대문구,1457688805.00000
...,...,...
225,합천군,433183384.00000
226,제주시,3600355259.00000
227,서귀포시,2445000131.00000
228,세종시,3822388136.00000


In [13]:
data4 = data3[['시도', '시군구', '합계']]

In [14]:
data4.to_excel('시군구_합계.xlsx') #편집으로 코드 다 붙여줌 나 정말 미쳐버리겠다~

In [25]:
data5 = pd.read_excel('시군구_합계_최종.xlsx')
data5

,SUM,CODE
0,4822165527,11680
1,1659002911,11740
2,933057384,11305
3,2752726888,11500
4,1735923906,11620
...,...,...
224,300610905,48870
225,433183384,48890
226,3600355259,50110
227,2445000131,50130


In [26]:
state_geo = 'sig_WGS84.json'

In [27]:
import json
json_data=open(state_geo, encoding='utf-8').read()
jsonResult = json.loads(json_data)

In [28]:
# 중앙위치 계산
def center_calc(points_df):
    x = points_df.x
    y = points_df.y

    X = (max(x)+min(x))/2.
    Y = (max(y)+min(y))/2.

    return X, Y

In [29]:
# 다중 Array 구조 이중으로 변환
def points_array(points):

    final_points = []

    for x in range(0, len(points)):

        if len(points[x]) == 2:
            final_points.append(points[x])
        else:
            target = points[x]
            for y in range(0, len(target)):
                final_points.append(target[y])

    return final_points

In [30]:
center_locations = pd.DataFrame()
codes = []
names = []
x_list = []
y_list = []
for x in range(0, len(jsonResult['features'])):
    code = jsonResult['features'][x]['properties']['SIG_CD']
    name = jsonResult['features'][x]['properties']['SIG_KOR_NM']
    # 중앙값 생성
    points = jsonResult['features'][x]['geometry']['coordinates'][0]
    points = points_array(points)
    points_df = pd.DataFrame(points)
    points_df.columns = ['x','y']
    X, Y = center_calc(points_df)

    # 결과
    codes.append(code)
    names.append(name)
    x_list.append(X)
    y_list.append(Y)

# 데이터 프레임 생성
center_locations['CODE'] = codes
center_locations['NAME'] = names
center_locations['X'] = x_list
center_locations['Y'] = y_list

In [31]:
center_locations

,CODE,NAME,X,Y
0,11110,종로구,126.98600,37.59900
1,11140,중구,126.99450,37.55800
2,11170,용산구,126.98150,37.53150
3,11200,성동구,127.04150,37.55050
4,11215,광진구,127.08550,37.54850
...,...,...,...,...
245,51790,화천군,127.67250,38.14100
246,51800,양구군,128.01200,38.16550
247,51810,인제군,128.24700,38.09600
248,51820,고성군,128.42500,38.38900


In [32]:
data4_r = data4.rename(columns={'시군구':'NAME'})


In [33]:
data5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229 entries, 0 to 228
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   SUM     229 non-null    int64
 1   CODE    229 non-null    int64
dtypes: int64(2)
memory usage: 3.7 KB


In [231]:
data5.to_excel('시군구_합계_최종.xlsx', index=False)

In [34]:
center_locations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   CODE    250 non-null    object 
 1   NAME    250 non-null    object 
 2   X       250 non-null    float64
 3   Y       250 non-null    float64
dtypes: float64(2), object(2)
memory usage: 7.9+ KB


In [35]:
data5.CODE = data5.CODE.astype(str) 
#state_data.CODE.dtype

In [36]:
data5 = data5.rename(columns={'시군구':'NAME', '합계':'SUM'})

In [38]:
target_df = pd.merge(data5,center_locations, how='left',on='CODE')
target_df

,SUM,CODE,NAME,X,Y
0,4822165527,11680,강남구,127.06650,37.49600
1,1659002911,11740,강동구,127.14600,37.54850
2,933057384,11305,강북구,127.01500,37.64700
3,2752726888,11500,강서구,126.82350,37.56550
4,1735923906,11620,관악구,126.94400,37.46600
...,...,...,...,...,...
224,300610905,48870,함양군,127.73150,35.54350
225,433183384,48890,합천군,128.16700,35.60000
226,3600355259,50110,제주시,126.22700,33.40850
227,2445000131,50130,서귀포시,126.26750,33.11800


In [39]:
import numpy as np
target_df = target_df[~np.isnan(target_df['X'])] # 정보 없는 값 제외

In [40]:
# Initialize the map:
m = folium.Map(location=[36, 127], zoom_start=7)


# I can add marker one by one on the map
for i in range(0,len(target_df)):
    latitude = target_df.iloc[i]['Y']
    longitude = target_df.iloc[i]['X']
    location=(latitude, longitude)
    folium.Circle(location, radius=target_df.iloc[i]['SUM']/600000,color='red',fill_color='red', popup=target_df.iloc[i]['NAME']).add_to(m)


folium.LayerControl(collapsed=False).add_to(m)



In [41]:
# Save to html
m.save(os.path.join('kr_energy.html'))
m
# webbrowser.open_new("kr_energy.html")